In [1]:
import os
import sys

os.getcwd()  # Check the current working directory
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..\..')))

import DifferentiationInterface.src.diffipy.diffipy as dp

In [2]:
backend_array = ['numpy', 'torch',  'tensorflow', 'jax']

for backend in backend_array:
    dp.setBackend(backend)
    # Initialize variables and constants
    s0_value = 100.0
    K_value = 110.0
    r_value = 0.05
    sigma_value = 0.3
    dt_value = 1.0
    z_value = 0.5
    N = 100000
    seed = 1

    # Define variables
    s0 = dp.variable(s0_value, 'input','s0')
    K = dp.variable(K_value, 'input','K')
    r = dp.variable(r_value, 'input','r')
    sigma = dp.variable(sigma_value, 'input','sigma')
    dt = dp.variable(dt_value, 'input','dt')
    z = dp.variable(z_value, 'randomVariableNormal','z')

    func_input_variables = [s0, K, r, sigma, dt, z]
    diff_variables = None#[s0, K, r, sigma, dt]

    # Record Tape: Standard Monte Carlo

    s = s0 * dp.exp((r - sigma **2 / 2) * dt + sigma * dp.sqrt(dt) * z)
    payoff =  dp.if_(s > K, s - K, 0)
    PV_standard = dp.exp(-r * dt) * dp.sum(payoff) / N

    ### Test performance
    dp.seed(seed)
    pre_computed_random_variables = z.NewSample(N) # pre-evaluate random samples

    PV_standard.run_performance_test(func_input_variables, diff_variables, warmup_iterations = 10, test_iterations = 40)


Backend              Eval-Result  mean runtime    variance runtime    gradient directions: {'dt', 'sigma', 'K', 's0', 'r'}
numpy                10.069236    0.005052        0.000048            [40.19209813019842, 8.025041059944726, 0.5026067643854049, 40.103230874777296, -0.36537673455683256]
numpy_as_func        10.069236    0.005262        0.000000            {'r': 40.1920981316195, 'dt': 8.025041063142169, 's0': 0.5026067581681559, 'sigma': 40.10323087761947, 'K': -0.3653767416622599}
torch                10.009336    0.003769        0.000001            [40.062740325927734, 7.97985315322876, 0.500720739364624, 39.84477233886719, -0.3642067313194275]
torch_as_func        10.009337    0.000750        0.000000            [tensor(40.0627), tensor(7.9799), tensor(0.5007), tensor(39.8448), tensor(-0.3642)]
tensorflow           9.987544     0.018524        0.000000            [40.1454, 7.9673195, 0.5013294, 39.733658, -0.36495823]
tensorflow_as_func   9.987547     0.004881        0.000000 